# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [1]:
import pandas as pd     # type: ignore
import numpy as np  # type: ignore
import sqlite3

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [2]:
conn = sqlite3.connect('go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [3]:
product = pd.read_sql_query("SELECT * FROM product", conn)
product_type = pd.read_sql_query("SELECT * FROM product_type", conn)
product_line = pd.read_sql_query("SELECT * FROM product_line", conn)
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", conn)
sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", conn)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", conn)
country = pd.read_sql_query("SELECT * FROM country", conn)
order_header = pd.read_sql_query("SELECT * FROM order_header", conn)
order_details = pd.read_sql_query("SELECT * FROM order_details", conn)
target = pd.read_sql_query("SELECT * FROM SALES_TARGETData", conn)
returned_item = pd.read_sql_query("SELECT * FROM returned_item", conn)
return_reason = pd.read_sql_query("SELECT * FROM return_reason", conn)

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [4]:
condition = (product['PRODUCTION_COST'].astype(float) < 100.0) & (product['PRODUCTION_COST'].astype(float) > 50.0)
product[condition][["PRODUCT_NAME", "PRODUCTION_COST"]]

,PRODUCT_NAME,PRODUCTION_COST
13,Blue Steel Putter,60
14,Blue Steel Max Putter,81.8
17,Course Pro Golf Bag,80
24,Hibernator Lite,60
25,Hibernator,86
28,Hibernator Self - Inflating Mat,78.55
31,Hibernator Camp Cot,65.33
32,Canyon Mule Climber Backpack,62.5
56,Granite Climbing Helmet,52.86
58,Husky Harness Extreme,55


Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [5]:
condition = (product['MARGIN'].astype(float) < .2) | (product['MARGIN'].astype(float) > .6)
product[condition][["PRODUCT_NAME", "MARGIN"]]

,PRODUCT_NAME,MARGIN
29,Hibernator Pad,.17
32,Canyon Mule Climber Backpack,.17
100,BugShield Natural,.67
101,BugShield Spray,.67
102,BugShield Lotion Lite,.7
103,BugShield Lotion,.63
105,BugShield Extreme,.63


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [6]:
currency_condition = country["CURRENCY_NAME"] == "francs"
country[currency_condition][["COUNTRY"]].sort_values("COUNTRY") # type: ignore

,COUNTRY
7,Belgium
0,France
19,Switzerland


Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [7]:
margin_greater_than_50_percent = product['MARGIN'].astype(float) > .5
product[margin_greater_than_50_percent][["INTRODUCTION_DATE"]].drop_duplicates()

,INTRODUCTION_DATE
4,10-12-2019
6,18-12-2019
7,27-12-2019
9,13-1-2020
14,15-12-2019
79,26-10-2016
100,15-2-2011


Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [8]:
# Het aantal regels klopt niet, maar volgens de docent is dat een fout van de opdracht.
condition = sales_branch["ADDRESS2"].notna() & sales_branch["REGION"].notna()
sales_branch.loc[condition][["ADDRESS1", "CITY"]]

,ADDRESS1,CITY
10,Prol. Paseo de la Reforma No. 51,Distrito Federal
11,202-2-3 Hyakunincho,Tokyo
12,543-225 Asahi,Osaka City
13,2315 Queen's Ave,Melbourne
15,"Avenida Paulista, 333",São Paulo
21,3 Albany Court,Birmingham
27,Singelgravenplein 4,Amsterdam


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [9]:
condition = country["CURRENCY_NAME"].str.contains("dollar")
country[condition].sort_values("COUNTRY")[["COUNTRY"]]  # type: ignore

,COUNTRY
6,Australia
15,Canada
3,Taiwan
14,United States


Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [10]:
condition = retailer_site["POSTAL_ZONE"].str.startswith("D") & retailer_site["ADDRESS2"].notna()
retailer_site[condition][["ADDRESS1", "ADDRESS2", "CITY"]]

,ADDRESS1,ADDRESS2,CITY
205,Röntgenstraße 90,3. Tür rechts,Frankfurt
351,Grubesallee 141,4. Stock,Hamburg


## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [11]:
returned_item.count()["RETURN_CODE"]

706

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [12]:
retailer_site.drop_duplicates().count()["REGION"]

251

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [13]:
data = pd.DataFrame(product["MARGIN"].astype(float).agg(["max", "min", "mean"]))
data.transpose()
data.T.round(2)

,max,min,mean
MARGIN,0.7,0.17,0.4


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [14]:
second_addr_unknown = retailer_site["ADDRESS2"].isna()
retailer_site[second_addr_unknown].count().iloc[0]    # type: ignore

286

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [15]:
got_discount = order_details["UNIT_SALE_PRICE"].astype(float) < order_details["UNIT_PRICE"].astype(float)
order_details.loc[got_discount, "UNIT_SALE_PRICE"].astype(float).mean()


177.99788862151942

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [16]:
pd.DataFrame(sales_staff.groupby("POSITION_EN", as_index=True)["POSITION_EN"].count())

,POSITION_EN
POSITION_EN,
Branch Manager,19
District Manager,4
General Manager,1
Level 1 Sales Representative,13
Level 2 Sales Representative,39
Level 3 Sales Representative,24
Regional Manager,2


Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [17]:
condition = sales_staff.groupby("WORK_PHONE", as_index=True)["WORK_PHONE"].count() > 4
pd.DataFrame(sales_staff.groupby("WORK_PHONE", as_index=True)["WORK_PHONE"].count()[condition])

,WORK_PHONE
WORK_PHONE,
+(44) 121 3505267,5
+(61) 03 2982 4242,5
+31 (0)20 692 93 94,5
+33 1 68 94 52 20,5
1 (206) 292-0012,5
1 (305) 557-4810,6
1 (310) 281-5722,5
1 (403) 232-5986,5
1 (416) 493-5595,5


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [18]:
stuff = pd.merge(retailer_site, country, on="COUNTRY_CODE")
condition = stuff["COUNTRY"] == "Netherlands"
stuff[condition][["ADDRESS1", "CITY"]]

,ADDRESS1,CITY
208,Beets 36,Beets
209,Perestraat 1,Deventer
210,Laan van Meerdervoort 966,Den Haag
211,Kalfjeslaan 47,Amstelveen
212,Startbaan 11,Amstelveen
213,Noorderspoorsingel 28,Groningen
214,Hogehilweg 13,Amsterdam
215,Spoorstraat 43,Varsseveld
216,Westhavenweg 8,Amsterdam
217,Westzeedijk 4,Muiden


Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [19]:
merged_df = pd.merge(product, product_type, on='PRODUCT_TYPE_CODE')
condition = merged_df['PRODUCT_TYPE_EN'] == 'Eyewear'
merged_df[condition][['PRODUCT_NAME']]

,PRODUCT_NAME
87,Polar Sun
88,Polar Ice
89,Polar Sports
90,Polar Wave
91,Polar Extreme


Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [20]:
condition = sales_staff["POSITION_EN"] == "Branch Manager"
pd.merge(pd.merge(retailer_site, order_header, on="RETAILER_SITE_CODE"), sales_staff[condition], on="SALES_STAFF_CODE")[["ADDRESS1", "FIRST_NAME", "LAST_NAME"]].drop_duplicates()

,ADDRESS1,FIRST_NAME,LAST_NAME
0,"Alameda Santos, 9876",Bayard,Lopes


Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [21]:
managers = sales_staff[sales_staff["POSITION_EN"].str.contains("Manager")]
pd.merge(managers, order_header, how='left', on='SALES_STAFF_CODE').loc[:, ["ORDER_DATE", "POSITION_EN"]].drop_duplicates()

,ORDER_DATE,POSITION_EN
0,NaN,Branch Manager
1,NaN,General Manager
4,NaN,Regional Manager
7,NaN,District Manager
25,2022-09-09,Branch Manager
26,2022-10-31,Branch Manager
27,2022-12-13,Branch Manager
28,2022-03-13,Branch Manager
29,2022-01-09,Branch Manager


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [22]:
stuff = pd.merge(pd.merge(product_type, product, on='PRODUCT_TYPE_CODE'), order_details, on="PRODUCT_NUMBER")
condition = stuff["QUANTITY"] > 750
stuff[condition][["PRODUCT_NAME", "PRODUCT_TYPE_EN"]].drop_duplicates()

,PRODUCT_NAME,PRODUCT_TYPE_EN
13769,BugShield Spray,Insect Repellents
14179,BugShield Lotion Lite,Insect Repellents
14636,BugShield Lotion,Insect Repellents
15024,BugShield Extreme,Insect Repellents
16160,Sun Shelter 15,Sunscreen
16512,Sun Shelter 30,Sunscreen
17087,Sun Shield,Sunscreen
23853,Star Peg,Tents
32669,Firefly 2,Lanterns


Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [23]:
stuff = pd.merge(product, order_details, on="PRODUCT_NUMBER")
condition = (stuff["UNIT_PRICE"].astype(float) - stuff["UNIT_SALE_PRICE"].astype(float)) / stuff["UNIT_PRICE"].astype(float) > 0.4
stuff[condition][["PRODUCT_NAME"]].drop_duplicates()

,PRODUCT_NAME
2765,Hailstorm Titanium Woods Set
37446,BugShield Natural
37967,BugShield Spray
38306,BugShield Lotion Lite
38772,BugShield Lotion
39561,BugShield Extreme
40799,Sun Shelter 15
41194,Sun Shelter 30


Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [24]:
stuff = pd.merge(pd.merge(returned_item, return_reason, on="RETURN_REASON_CODE"), order_details, on="ORDER_DETAIL_CODE")
condition = stuff["RETURN_QUANTITY"].astype(float) / stuff["QUANTITY"].astype(float) > 0.9
stuff[condition][["RETURN_DESCRIPTION_EN"]].drop_duplicates()

,RETURN_DESCRIPTION_EN
0,Unsatisfactory product
332,Wrong product shipped
394,Wrong product ordered


## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [25]:
pd.DataFrame(product.merge(product_type, on="PRODUCT_TYPE_CODE").groupby("PRODUCT_TYPE_EN").size())

,0
PRODUCT_TYPE_EN,
Binoculars,4
Climbing Accessories,7
Cooking Gear,10
Eyewear,5
First Aid,5
Golf Accessories,4
Insect Repellents,5
Irons,4
Knives,5


Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [26]:
pd.DataFrame(pd.merge(retailer_site, country, on='COUNTRY_CODE').groupby('COUNTRY').size())

,0
COUNTRY,
Australia,10
Austria,10
Belgium,10
Brazil,1
Canada,31
China,10
Denmark,6
Finland,5
France,29


Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [27]:
stuff = pd.merge(pd.merge(product, product_type, on="PRODUCT_TYPE_CODE"), order_details, on="PRODUCT_NUMBER")
condition = stuff["PRODUCT_TYPE_EN"] == "Cooking Gear"
stuff.loc[:, :] = stuff[condition]
stuff.loc[:, ["TOTAAL_VERKOCHTE_HOEVEELHEID"]] = stuff["QUANTITY"].sum()
stuff.loc[:, ["GEMIDDELDE_VERKOOPPRIJS"]] = (stuff["UNIT_PRICE"].astype(float) + stuff["UNIT_SALE_PRICE"].astype(float)).astype(float) / 2


result = stuff[["PRODUCT_NAME", "GEMIDDELDE_VERKOOPPRIJS", "TOTAAL_VERKOCHTE_HOEVEELHEID"]].drop_duplicates()

result.groupby("PRODUCT_NAME")[["TOTAAL_VERKOCHTE_HOEVEELHEID", "GEMIDDELDE_VERKOOPPRIJS"]].first()

,TOTAAL_VERKOCHTE_HOEVEELHEID,GEMIDDELDE_VERKOOPPRIJS
PRODUCT_NAME,,
TrailChef Canteen,198676.0,12.13
TrailChef Cook Set,198676.0,54.09
TrailChef Cup,198676.0,7.15
TrailChef Deluxe Cook Set,198676.0,123.56
TrailChef Double Flame,198676.0,134.63
TrailChef Kettle,198676.0,12.15
TrailChef Kitchen Kit,198676.0,22.84
TrailChef Single Flame,198676.0,65.23
TrailChef Utensils,198676.0,17.40


Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

In [28]:
# Afhankelijk van of de verkoopafdeling in "sales_branch" of "retailer_site" moet zijn, zijn er 2 mogelijke uitwerkingen. Hieronder ziet u ze allebei.
# Omdat retailer_site waarschijnlijk klantgegevens bevat (zie analyse in de volgende opgave) denk ik dat ik sales_branch moet gebruiken.
all_data = pd.merge(
    pd.merge(target, sales_staff, on="SALES_STAFF_CODE"),
    sales_branch,
    on="SALES_BRANCH_CODE"
)

condition = all_data["CITY"] == "Amsterdam"

all_data = all_data[condition]

grouped_data = all_data.groupby(["FIRST_NAME", "LAST_NAME"])

all_data["MAX_SALES_TARGET"] = grouped_data["SALES_TARGET"].transform("max")
all_data["MIN_SALES_TARGET"] = grouped_data["SALES_TARGET"].transform("min")

all_data[condition][["FIRST_NAME", "LAST_NAME", "MIN_SALES_TARGET", "MAX_SALES_TARGET"]].drop_duplicates()

# all_data = pd.merge(
#     pd.merge(target, sales_staff, on="SALES_STAFF_CODE"),
#     retailer_site,
#     on="RETAILER_CODE"
# )

# condition = all_data["CITY"] == "Amsterdam"

# all_data = all_data[condition]

# grouped_data = all_data.groupby(["FIRST_NAME", "LAST_NAME"])

# all_data["MAX_SALES_TARGET"] = grouped_data["SALES_TARGET"].transform("max")
# all_data["MIN_SALES_TARGET"] = grouped_data["SALES_TARGET"].transform("min")

# all_data[condition][["FIRST_NAME", "LAST_NAME", "MIN_SALES_TARGET", "MAX_SALES_TARGET"]].drop_duplicates()

C:\Users\Laurens\AppData\Local\Temp\ipykernel_15532\742529626.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  all_data[condition][["FIRST_NAME", "LAST_NAME", "MIN_SALES_TARGET", "MAX_SALES_TARGET"]].drop_duplicates()


,FIRST_NAME,LAST_NAME,MIN_SALES_TARGET,MAX_SALES_TARGET
5467,Belinda,Jansen-Velasquez,1004,984
5827,Ellen,Shapiro,100,997
6376,Jan,Haverkamp,1001,9987
6875,Maria,Laponder,1000,9993


Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [29]:
# Deze opdracht is vaag. De verkoopafdeling kan "sales_branch" of "retailer_site" betreffen, beide tabellen hebben een "city".
# De verkoopafdeling moet ook een city hebben, die hebben beide tabellen.

# Na heel wat brainstormen heb ik het volgende vastgesteld:
# order_header lijkt klantgegevens te betreffen
# die heeft een vreemde sleutel naar retailer_site, wat dus waarschijnlijk ook klantgegevens zullen betreffen
# Dan moet sales_branch dus de verkoopafdeling zijn

# Feedback: Naar mijn idee is dit teveel deductiewerk voor een schoolopdracht, ik zou het erg op prijs stellen als dit de volgende keer wat duidelijker is.
extended_sales_branch = pd.merge(country, sales_branch, on="COUNTRY_CODE")
extended_retailer_site = pd.merge(country, retailer_site, on="COUNTRY_CODE")

extended_sales_branch = extended_sales_branch.rename({
    "CITY": "klanten"
}, axis=1)

extended_retailer_site = extended_retailer_site.rename({
    "CITY": "verkoper"
}, axis=1)

all_data = pd.merge(extended_sales_branch, extended_retailer_site, on="COUNTRY_CODE")

all_data[["COUNTRY_x", "verkoper", "klanten"]].groupby(["COUNTRY_x", "verkoper"])["klanten"].count()

COUNTRY_x      verkoper   
Australia      Brisbane       1
               Cairns         2
               Melbourne      2
               NewCastle      2
               Perth          1
                             ..
United States  Seattle        8
               Springfield    4
               Topeka         4
               Tucson         4
               Washington     4
Name: klanten, Length: 229, dtype: int64

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [30]:
output_data = []

for index, staff_row in sales_staff.iterrows():

    sales_staff_code = staff_row["SALES_STAFF_CODE"]

    for index, order_row in order_header.iterrows():
        if order_row["SALES_STAFF_CODE"] == sales_staff_code:
            break
    else:
        output_data.append([staff_row["FIRST_NAME"], staff_row["LAST_NAME"]])


pd.DataFrame(output_data)


,0,1
0,Walter,Taylor
1,Penelope,Tamrine
2,Polly,Linton
3,Gracy,Gellens
4,Else,Mörike
5,Frank,Fuchs
6,Fritz,Hirsch
7,Jörg,Kunze
8,Maria,Iacobucci
9,Kick,Kalkman


Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [31]:
# Gemiddelde marge berekenen
totale_marge = 0.0
aantal = 0
for index, product_row in product.iterrows():
    totale_marge += float(product_row["MARGIN"])
    aantal += 1

gemiddelde_marge = totale_marge / aantal

# Aantal producten met een lage marge berekenen
aantal = 0
totale_marge = 0.0
for index, product_row in product.iterrows():
    marge = float(product_row["MARGIN"])
    if marge < gemiddelde_marge:
        aantal += 1
        totale_marge += marge

gemiddelde_lage_marge = totale_marge / aantal

pd.DataFrame([aantal, gemiddelde_lage_marge])

,0
0,59.000000
1,0.292203


Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [32]:
# Producten die meer dan 500 kosten
condition = order_details["UNIT_PRICE"].astype(float) > 500
filtered_order_details = order_details[condition]

# Producten die meer dan 500 kosten samenvoegen met de teruggestuurde items
merged_df = pd.merge(filtered_order_details, returned_item, on='ORDER_DETAIL_CODE', how='left', indicator=True)

# De stuff die alleen in de gefilterde lijst stond, en niet in de returned_item lijst, die is zowel duurder dan 500 piek, als nooit teruggebracht
not_returned = merged_df[merged_df['_merge'] == 'left_only']

# De naam van het product stond niet in de tabellen die we hebben gemerged, dus mergen we met product
final_df = pd.merge(not_returned, product, on='PRODUCT_NUMBER')

# En halen we de dubbelen weg
found_products = final_df['PRODUCT_NAME'].unique()

# En tonen we het resultaat
pd.DataFrame(found_products)

# PS. oorspronkelijk had ik hier een O(n^2) algoritme gebruikt wat 2 minuten kost om te draaien met een print-statement dat u beter een bakje koffie kunt halen

,0
0,Hailstorm Steel Irons
1,Hailstorm Titanium Irons
2,Lady Hailstorm Steel Irons
3,Lady Hailstorm Titanium Irons
4,Hailstorm Titanium Woods Set
5,Hailstorm Steel Woods Set
6,Lady Hailstorm Titanium Woods Set
7,Lady Hailstorm Steel Woods Set
8,Star Dome
9,Star Gazer 2


Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [33]:
pd.DataFrame([
    [row["LAST_NAME"], "Ja" if "Manager" in row["POSITION_EN"] else "Nee"]
    for _, row in sales_staff.iterrows()
])


,0,1
0,Savolainen,Nee
1,Kim,Nee
2,Choi,Nee
3,Kao,Nee
4,Fang,Nee
...,...,...
97,Iglesias,Nee
98,Lopes,Ja
99,Barros,Nee
100,Guimarães,Nee


Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [34]:
from datetime import date
date.today().year

2024

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [35]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

In [36]:
# sales_staff (sales_staff_code, first_name, last_name, position_en, work_phone, extension, fax, email, date_hired, sales_branch_code)

pd.DataFrame([
    [row["FIRST_NAME"], row["LAST_NAME"] + " is " + ("kort" if (datetime.now() - datetime.strptime(row["DATE_HIRED"], "%d-%m-%Y")).days / 365.25 < 12 else "lang") + " in dienst"]
    for _, row in sales_staff.iterrows()
])

,0,1
0,Tuomas,Savolainen is kort in dienst
1,Chang-ho,Kim is kort in dienst
2,Jung-ho,Choi is kort in dienst
3,Kwei-Ping,Kao is kort in dienst
4,Chin-Tsai,Fang is kort in dienst
...,...,...
97,Tomás,Iglesias is kort in dienst
98,Bayard,Lopes is kort in dienst
99,Viviam,Barros is kort in dienst
100,Eduardo,Guimarães is kort in dienst
